# Introduction to WaterTAP Costing

## Overview

This tutorial will demonstrate:

- How to add costing on a flowsheet for reverse osmosis and pump unit models
- How to calculate system-level cost metrics (LCOW, SEC)
- How to access and update default cost parameters

## Step 1: Import Modules

In [ ]:
# Imports from Pyomo
from pyomo.environ import (
    ConcreteModel,
    Var,
    Constraint,
    Objective,
    value,
    assert_optimal_termination,
    TransformationFactory,
    units as pyunits,
)

# Imports from IDAES
from idaes.core import FlowsheetBlock


# Imports from WaterTAP
from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.unit_models.pressure_changer import Pump
from watertap.unit_models.reverse_osmosis_0D import (
    ReverseOsmosis0D,
    ConcentrationPolarizationType,
    MassTransferCoefficient,
    PressureChangeType,
)

from watertap.core.solvers import get_solver

In [ ]:
from watertap.costing import WaterTAPCosting
from idaes.core import UnitModelCostingBlock

## Step 2: Create an RO Model

In [ ]:
# Build flowsheet essentials
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

# Build property model
m.fs.properties = SeawaterParameterBlock()

# Build unit model
m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    has_pressure_change=True,
    pressure_change_type=PressureChangeType.calculated,
    mass_transfer_coefficient=MassTransferCoefficient.calculated,
    concentration_polarization_type=ConcentrationPolarizationType.calculated,
)

## Step 3: Add Costing

### Step 3a: Add Costing Block to the Flowsheet

The `WaterTAPCosting` block:
- Contains methods, variables, and constraints for **flowsheet** level cost metrics
- Aggregates unit model specific capital and operating costs for all units on the **flowsheet**
- Is conventionally defined as `m.fs.costing`

We'll also establish the base currency for the cost model.

<p align="center">
<img src='graphics/cost_framework_1.png' alt="Flowsheet Costing Framework" width="400">
</p>

In [ ]:
# Set up WaterTAP Costing
m.fs.costing = WaterTAPCosting()

# Display flowsheet costing block components
m.fs.costing.display()

In [ ]:
# Print default base currency
print("Default base currency:", m.fs.costing.base_currency)

# Set units of base currency
m.fs.costing.base_currency = pyunits.USD_2020

# Print updated base currency
print("Updated base currency:", m.fs.costing.base_currency)

### Step 3b: Add RO Unit Model Costing Block

The `UnitModelCostingBlock` is used to add unit-specific costing to the flowsheet.

**What it does:**
1. Adds unit-specific costing variables and constraints to the flowsheet costing block. In our example it is added as `m.fs.costing.reverse_osmosis`
2. Creates a costing block on the unit model with capital and operating cost variables. In our example it is added as `m.fs.RO.costing`
3. Registers flows (electricity, chemicals) with the flowsheet costing block

**Required Configuration:**
- `flowsheet_costing_block`: Reference to the flowsheet-level costing block (usually `m.fs.costing`)

**Note:** Variable operating costs (electricity, chemicals) are aggregated at the flowsheet level.

<p align="center">
<img src='graphics/cost_framework_2.png' alt="Unit Model Costing Framework" width="400">
</p>

In [ ]:
# Create RO costing block
m.fs.RO.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
)

### Step 3c: Add Pump Unit Model Costing Block

In [ ]:
# Build pump unit model
m.fs.high_pressure_pump = Pump(
    property_package=m.fs.properties,
)

# Add pump UnitModelCostingBlock
m.fs.high_pressure_pump.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
)

## Display the Flowsheet Costing Block Components

Reverse osmosis and pump cost variables are now added to the flowsheet costing block `m.fs.costing` and can be accessed as shown below.

**Key Points:**

- All instances of the reverse osmosis/pump model on the flowsheet share the same global parameters
- The block name `reverse_osmosis`/`high_pressure_pump` is defined in the RO/pump costing model and predefined

In [ ]:
m.fs.costing.display()

## Step 4: Add System Level Cost Metrics

The `cost_process()` method creates variables for system-level cost metrics by aggregating costs across all unit models:

**Aggregate Costs:**
- `aggregate_capital_cost` - Sum of all unit capital costs
- `aggregate_fixed_operating_cost` - Sum of all fixed operating costs
- `aggregate_variable_operating_cost` - Sum of all variable operating costs
- `aggregate_flow_costs` - Costs for electricity, chemicals, etc.

**Total Costs:**
- `total_operating_cost` - Total annual operating cost
- `total_capital_cost` - Total annualized capital cost

<p align="center">
<img src='graphics/cost_framework_3.png' alt="Pump Costing Assumptions" width="1000">
</p>

In [ ]:
# Create aggregate and total costing variables by using cost_process
m.fs.costing.cost_process()

## Step 5: Add Levelized Cost of Water (LCOW) and Specific Energy Consumption (SEC)

The required argument for both functions is the volumetric flowrate for which they are calculated

In [ ]:
# Add levelized cost of water
m.fs.costing.add_LCOW(m.fs.RO.mixed_permeate[0].flow_vol_phase["Liq"])

# Add specific energy consumption (SEC)
m.fs.costing.add_specific_energy_consumption(
    m.fs.RO.mixed_permeate[0].flow_vol_phase["Liq"],
)

## Step 6: Solve Model with Costing

In [ ]:
# Initialize costing block
m.fs.costing.initialize()

# Solve the model
solver = get_solver()
results = solver.solve(m)

## Accessing Aggregated Costs Metrics

In [ ]:
# Display costing results
print("\nCosting Results:")

# Print aggregated capital cost for all unit models on the flowsheet
print(
    f"Aggregate Capital Cost:", value(m.fs.costing.aggregate_capital_cost), "USD/year"
)

# Print aggregated operating cost for all unit models on the flowsheet
print(
    f"Aggregate Operating Cost:",
    value(m.fs.costing.aggregate_fixed_operating_cost),
    "USD/year",
)

# Print aggregated variable operating cost for all unit models on the flowsheet
print(
    f"Aggregate Variable Operating Cost:",
    value(m.fs.costing.aggregate_variable_operating_cost),
    "USD/year",
)

# Print the aggregate electricity cost
print(
    "Aggregate electricity cost:",
    value(m.fs.costing.aggregate_flow_costs["electricity"]),
    "USD_2020/year",
)

# Print the aggregate electricity flow
print(
    "Aggregate electricity flow:",
    value(m.fs.costing.aggregate_flow_electricity),
    pyunits.get_units(m.fs.costing.aggregate_flow_electricity),
)

## Accessing Total Costs

In [ ]:
# Print total capital cost for all unit models on the flowsheet
print(f"Total Capital Cost:", value(m.fs.costing.total_capital_cost), "USD/year")

# Print total operating cost for all unit models on the flowsheet
print(f"Total Operating Cost:", value(m.fs.costing.total_operating_cost), "USD/year")

# Print total fixed operating cost for all unit models on the flowsheet
print(
    f"Total Fixed Operating Cost:",
    value(m.fs.costing.total_fixed_operating_cost),
    "USD/year",
)

# Print total variable operating cost for all unit models on the flowsheet
print(
    f"Total Variable Operating Cost:",
    value(m.fs.costing.total_variable_operating_cost),
    "USD/year",
)

In [ ]:
# Print flowsheet levelized cost of water
print(f"Levelized Cost of Water (LCOW):", value(m.fs.costing.LCOW), "USD/m3")

# Print specific cost of energy
print(
    f"Specific Energy Consumption (SEC):",
    value(m.fs.costing.specific_energy_consumption),
    "kWh/m3",
)

## Modify Existing Parameters

You can modify default cost parameters by fixing them to new values.

**Example:** Change the membrane unit cost from 30 USD/m<sup>2</sup> to 60 USD/m<sup>2</sup>

The cost variable `membrane_cost` is located in `m.fs.costing.reverse_osmosis` (added by the unit model cost block).

In [ ]:
# Print default RO membrane cost
print(
    "Default RO membrane cost:",
    value(m.fs.costing.reverse_osmosis.membrane_cost),
    "USD/m2",
)

# Update RO membrane cost
m.fs.costing.reverse_osmosis.membrane_cost.fix(60)

# Print updated RO membrane cost
print(
    "Updated RO membrane cost:",
    value(m.fs.costing.reverse_osmosis.membrane_cost),
    "USD/m2",
)

## Try It Yourself

**Task:** Update the electricity cost to 0.2 $/kWh and observe how it affects the total operating cost.

<details>
  <summary>Click the arrow for hint #1!</summary>
    
The electricity cost variable is on the flowsheet costing block and can be updated by fixing `m.fs.costing.electricity_cost`
</details>

<details>
  <summary>Click the arrow for hint #2!</summary>
    
The aggregate electricity cost can be printed using `m.fs.costing.aggregate_flow_electricity`
</details>

In [ ]:
# Update the electricity cost in the flowsheet costing block m.fs.costing


# Initialize costing and Re-solve


# Print aggregate electricity cost results again

## Summary

This tutorial covered the essential steps for adding costing to a WaterTAP flowsheet:

### Key Steps:

1. **Import Modules** - Import WaterTAP costing components (`WaterTAPCosting`, `UnitModelCostingBlock`)

2. **Create Model** - Build your flowsheet with unit models (RO, Pump, etc.)

3. **Add Flowsheet-Level Costing**
   - Create `WaterTAPCosting` block: `m.fs.costing = WaterTAPCosting()`
   - Set base currency: `m.fs.costing.base_currency = pyunits.USD_2020`

4. **Add Unit-Level Costing**
   - Add `UnitModelCostingBlock` to each unit: `m.fs.unit.costing = UnitModelCostingBlock(flowsheet_costing_block=m.fs.costing)`
   - This creates unit-specific cost variables and registers flows

5. **Modify Parameters** - Update default cost parameters using `.fix()`: 
   - `m.fs.costing.reverse_osmosis.membrane_cost.fix(60)`

6. **Add System Metrics**
   - Call `m.fs.costing.cost_process()` to create aggregate variables
   - Add metrics: `m.fs.costing.add_LCOW()`, `m.fs.costing.add_specific_energy_consumption()`
   - Initialize the costing model

7. **Solve Model** - Solve with costing constraints included

8. **Access Results** - View capital costs, operating costs, LCOW, SEC, and flow costs

### Key Concepts:

- **Flowsheet-level costing** (`m.fs.costing`) aggregates costs from all units
- **Unit-level costing** (`m.fs.unit.costing`) contains unit-specific cost variables
- **Global parameters** (e.g., `m.fs.costing.reverse_osmosis.membrane_cost`) are shared across all instances of that unit type
- **Variable operating costs** (electricity, chemicals) are tracked and aggregated at the flowsheet level